In [5]:
from __future__ import absolute_import, division, print_function
import os
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pypinyin import lazy_pinyin
from sklearn.preprocessing import LabelEncoder
import time
import datetime

In [13]:
class Configure(object):
    base_path = '../'

    # 数据清洗后的路径
    cleaned_path = base_path + 'cleaned/'
    # 生成的特征的路径
    features_path = base_path + 'features/'
    # 生成的模型可训练和预测的数据集
    datasets_path = base_path + 'datasets/'
    #
    raw_data_path = base_path + 'processed_data/'

    # 待 merge 的特征（特征名：merge_on 的特征）
    features = {
        'basic_user_info'             : {'on': 'userid', 'how': 'left'},
        'basic_user_action_features'  : {'on': 'userid', 'how': 'left'},
        'user_order_history_features' : {'on': 'userid', 'how': 'left'},
        'user_order_history_features2': {'on': 'userid', 'how': 'left'},
        'user_order_history_features3': {'on': 'userid', 'how': 'left'},
        'user_order_history_features4': {'on': 'userid', 'how': 'left'},
        'user_order_comment_features' : {'on': 'userid', 'how': 'left'},
        'action_history_features'     : {'on': 'userid', 'how': 'left'},
        'action_history_features2'    : {'on': 'userid', 'how': 'left'},
        'action_history_features3'    : {'on': 'userid', 'how': 'left'},
        'action_history_features4'    : {'on': 'userid', 'how': 'left'},
        'action_history_features5'    : {'on': 'userid', 'how': 'left'},
        'action_history_features6'    : {'on': 'userid', 'how': 'left'},
        # 'action_history_features7'   : {'on': 'userid', 'how': 'left'},
        'action_history_features8'    : {'on': 'userid', 'how': 'left'},
        'action_history_features9'    : {'on': 'userid', 'how': 'left'},
        'action_history_features10'   : {'on': 'userid', 'how': 'left'},
        'action_history_features11'   : {'on': 'userid', 'how': 'left'},
        # 'action_history_features12'   : {'on': 'userid', 'how': 'left'},
        # 'baseline_features'          : {'on': 'userid', 'how': 'left'},
        'wxr_features'                : {'on': 'userid', 'how': 'left'},
    }

    # 新提取的特征
    new_features = {
        'advance_order_history_features'    : {'on': 'userid', 'how': 'left'},
        'advance_action_features1'          : {'on': 'userid', 'how': 'left'},
        'other_features1'                   : {'on': 'userid', 'how': 'left'},
        'action_order_features1'            : {'on': 'userid', 'how': 'left'},
        'action_order_features2'            : {'on': 'userid', 'how': 'left'},
    }

    submit_result_path = '../result/'

    default_start_order_time = int(time.mktime(datetime.datetime(2016, 11, 1).timetuple()))
    default_end_order_time = int(time.mktime(datetime.datetime(2017, 9, 12).timetuple()))


In [30]:
def is_feature_created(feature_name):
    feature_files = [f for f in os.listdir(Configure.features_path) if os.path.isfile(os.path.join(Configure.features_path, f))]
    exit_feature = sum([feature_name in feature for feature in feature_files]) > 0
    return exit_feature

In [12]:
feature_name = 'basic_user_info'
is_feature_created(feature_name)
#true说明处理过了

False

In [14]:
train_user = pd.read_csv(Configure.raw_data_path + 'train/userProfile_train.csv', encoding='utf8')
test_user = pd.read_csv(Configure.raw_data_path + 'test/userProfile_test.csv', encoding='utf8')

In [17]:
#性别 
def gender_convert(gender):
    if gender == gender:
        return 'man' if gender == u'男' else 'woman'
    return 'None'


In [18]:
train_user['gender'] = train_user['gender'].map(gender_convert)
test_user['gender'] = test_user['gender'].map(gender_convert)
dummies = pd.get_dummies(train_user['gender'], prefix='gender')
train_user[dummies.columns] = dummies
dummies = pd.get_dummies(test_user['gender'], prefix='gender')
test_user[dummies.columns] = dummies

In [19]:
#省份
def province_convert(province):
    if province == province:
        return '_'.join(lazy_pinyin(province))
    return 'None'


In [20]:
train_user['province'] = train_user['province'].map(province_convert)
test_user['province'] = test_user['province'].map(province_convert)
le = LabelEncoder()
le.fit(train_user['province'].values)
train_user['province_code'] = le.transform(train_user['province'])
test_user['province_code'] = le.transform(test_user['province'])

In [21]:
#年龄段

train_user['age'] = train_user['age'].map(lambda age: 'lg' + age[:2] if age == age else 'None')
test_user['age'] = test_user['age'].map(lambda age: 'lg' + age[:2] if age == age else 'None')


In [22]:
#save cleaned datasets
train_user.to_csv(Configure.cleaned_path + 'cleaned_userProfile_train.csv', index=False, columns=train_user.columns)
test_user.to_csv(Configure.cleaned_path + 'cleaned_userProfile_test.csv', index=False, columns=test_user.columns)

In [23]:
dummies = pd.get_dummies(train_user['age'], prefix='age')
train_user[dummies.columns] = dummies
dummies = pd.get_dummies(test_user['age'], prefix='age')
test_user[dummies.columns] = dummies

In [25]:
import _pickle as cPickle
def save_features(train, test, features_name):
    if train is not None:
        train_path = Configure.features_path + 'train_' + features_name + '.pkl'
        with open(train_path, "wb") as f:
            cPickle.dump(train, f, -1)

    if test is not None:
        test_path = Configure.features_path + 'test_' + features_name + '.pkl'
        with open(test_path, "wb") as f:
            cPickle.dump(test, f, -1)


In [26]:
save_features(train_user, test_user, features_name=feature_name)

In [36]:
feature_name = 'basic_user_action_features'
is_feature_created(feature_name)

False

In [37]:
train_action = pd.read_csv(Configure.raw_data_path + 'train/action_train.csv')
test_action = pd.read_csv(Configure.raw_data_path + 'test/action_test.csv')


In [38]:
def build_time_features(action_df):

    action_df['actionTime'] = pd.to_datetime(action_df['actionTime'], unit='s')
    # 训练集和测试集最后一天是 2017-09-11
    now = datetime.datetime(2017, 9, 12)
    action_df['days_from_now'] = action_df['actionTime'].map(lambda order: (now - order).days)

    action_df['action_year'] = action_df['actionTime'].dt.year
    action_df['action_month'] = action_df['actionTime'].dt.month
    action_df['action_day'] = action_df['actionTime'].dt.day
    action_df['action_weekofyear'] = action_df['actionTime'].dt.weekofyear
    action_df['action_weekday'] = action_df['actionTime'].dt.weekday
    action_df['action_hour'] = action_df['actionTime'].dt.hour
    action_df['action_minute'] = action_df['actionTime'].dt.minute
    action_df['action_is_weekend'] = action_df['action_weekday'].map(lambda d: 1 if (d == 0) | (d == 6) else 0)
    action_df['action_week_hour'] = action_df['action_weekday'] * 24 + action_df['action_hour']

    def action_type_convert(action):
        if action == 1:
            return 'open_app'
        elif action == 2:
            return 'browse_product'
        elif action == 3:
            return 'browse_product2'
        elif action == 4:
            return 'browse_product3'
        elif action == 5:
            return 'fillin_form5'
        elif action == 6:
            return 'fillin_form6'
        elif action == 7:
            return 'fillin_form7'
        elif action == 8:
            return 'submit_order'
        elif action == 9:
            return 'pay_money'

    # action 操作，合并无序的浏览特征
    action_df['actionType'] = action_df['actionType'].map(action_type_convert)

    return action_df


In [39]:
train_action = build_time_features(train_action)
test_action = build_time_features(test_action)

In [40]:
train_action.to_csv(Configure.cleaned_path + 'cleaned_action_train.csv', index=False, columns=train_action.columns)
test_action.to_csv(Configure.cleaned_path + 'cleaned_action_test.csv', index=False, columns=test_action.columns)

In [41]:
def basic_action_info(action_df):
    """
    用户行为信息
    """
    print('用户不同操作的购买率')
    action_features = action_df.groupby(['userid', 'actionType']).actionTime.count().groupby(level=0).apply(lambda x: x.astype(float) / x.sum()).reset_index()
    action_features = action_features.pivot('userid', 'actionType', 'actionTime').reset_index().fillna(0)

    action_features = action_features.merge(
        action_df.groupby(['userid']).count().reset_index()[['userid', 'actionTime']].rename(columns={'actionTime': 'action_counts'}),
        on='userid', how='left'
    )
    action_features.rename(columns={'open_app': 'open_app_ratio', 'browse_product': 'browse_product_ratio',
                                    'browse_product2': 'browse_product2_ratio', 'browse_product3': 'browse_product3_ratio',
                                    'fillin_form5': 'fillin_form5_ratio', 'fillin_form6': 'fillin_form6_ratio',
                                    'fillin_form7': 'fillin_form7_ratio', 'submit_order': 'submit_order_ratio',
                                    'pay_money': 'pay_money_ratio'}, inplace=True)
    action_features['has_pay_money'] = (action_features['pay_money_ratio'] > 0).astype(int)

    print('拉普拉斯平滑计算基本的转化率')
    action_features['open_app_pay_money_ratio'] = (action_features['pay_money_ratio']+ 0.1) / (action_features['open_app_ratio']+ 0.2)
    action_features['browse_product_pay_money_ratio'] = (action_features['pay_money_ratio']+ 0.1) / (action_features['browse_product_ratio']+ 0.2)
    action_features['browse_product2_pay_money_ratio'] = (action_features['pay_money_ratio']+ 0.1) / (action_features['browse_product2_ratio']+ 0.2)
    action_features['browse_product3_pay_money_ratio'] = (action_features['pay_money_ratio']+ 0.1) / (action_features['browse_product3_ratio']+ 0.2)
    action_features['fillin_form5_pay_money_ratio'] = (action_features['pay_money_ratio']+ 0.1) / (action_features['fillin_form5_ratio']+ 0.2)
    action_features['fillin_form6_pay_money_ratio'] = (action_features['pay_money_ratio']+ 0.1) / (action_features['fillin_form6_ratio']+ 0.2)
    action_features['fillin_form7_pay_money_ratio'] = (action_features['pay_money_ratio']+ 0.1) / (action_features['fillin_form7_ratio']+ 0.2)
    action_features['submit_order_pay_money_ratio'] = (action_features['pay_money_ratio']+ 0.1) / (action_features['submit_order_ratio']+ 0.2)

    print('每年 action 的情况')
    action_features = action_features.merge(
        action_df.groupby(['userid', 'action_year']).count().reset_index()[['userid', 'action_year', 'actionTime']] \
            .pivot('userid', 'action_year', 'actionTime').reset_index().fillna(0).rename(
            columns={2016: '2016_year_pay_money_count', 2017: '2017_year_pay_money_count'}),
        on='userid', how='left'
    )
    action_features['year_action_count'] = action_features['userid'].map(lambda userid: len(action_df[action_df['userid'] == userid]['action_year'].unique()))

    print('每月 action 的情况')
    action_features = action_features.merge(
        action_df.groupby(['userid', 'action_month']).count().reset_index()[['userid', 'action_month', 'actionTime']] \
            .pivot('userid', 'action_month', 'actionTime').reset_index().fillna(0).rename(columns={i: 'month_{}_count'.format(i) for i in range(1, 13)}),
        on='userid', how='left'
    )
    action_features['month_action_count'] = action_features['userid'].map(lambda userid: len(action_df[action_df['userid'] == userid]['action_month'].unique()))

    print('最清闲的在几月，以及 action 的次数')

    def get_most_free_month(row):
        most_count = 0
        most_month = -1
        for i in range(1,13):
            if row['month_{}_count'.format(i)] > most_count:
                most_month = i
        return most_month

    action_features['most_free_month'] = action_features.apply(lambda row: get_most_free_month(row), axis=1)
    action_features['most_free_month_action_count'] = action_features.apply(lambda row: row['month_{}_count'.format(int(row['most_free_month']))], axis=1)

    return action_features

In [42]:
train_action_features = basic_action_info(train_action)
test_action_features = basic_action_info(test_action)

用户不同操作的购买率
拉普拉斯平滑计算基本的转化率
每年 action 的情况
每月 action 的情况
最清闲的在几月，以及 action 的次数
用户不同操作的购买率
拉普拉斯平滑计算基本的转化率
每年 action 的情况
每月 action 的情况
最清闲的在几月，以及 action 的次数


In [43]:
save_features(train_action_features, test_action_features, features_name=feature_name)

In [44]:
train = pd.read_csv(Configure.raw_data_path + 'train/orderFuture_train.csv', encoding='utf8', dtype={"userid":str})
test = pd.read_csv(Configure.raw_data_path + 'test/orderFuture_test.csv', encoding='utf8', dtype={"userid":str})
orderHistory_train = pd.read_csv(Configure.raw_data_path + 'train/orderHistory_train.csv', encoding='utf8', dtype={"userid":str})
orderHistory_test = pd.read_csv(Configure.raw_data_path + 'test/orderHistory_test.csv', encoding='utf8', dtype={"userid":str})

In [50]:
from tqdm import tqdm
def order_last_num(order):
    """ 按时间倒序对订单排序 """
    users = list(set(order['userid']))
    order_c = order.copy()
    order_c['order_number'] = 1
    for i in tqdm(range(len(users))):
        slit_df = order_c[order_c['userid'] == users[i]]
        order_c.loc[slit_df.index, 'order_number'] = range(slit_df.shape[0],0,-1)
    return order_c

In [53]:
def days_since_prior_order(order):
    """ 用户两次订单之间的时间间隔 """
    users = list(set(order['userid']))
    order_c = order.copy()
    order_c['days_since_prior_order'] = np.nan
    for i in tqdm(range(len(users))):
        slit_df = order_c[order_c['userid'] == users[i]]
        time_shift = slit_df['orderTime'].shift(1)
        time_series = pd.Series(slit_df['orderTime'].values - time_shift.values).map(lambda x: x/np.timedelta64(1, 's'))/(24*3600.0)
        order_c.loc[slit_df.index, 'days_since_prior_order'] = time_series.values
    return order_c

In [54]:
def build_time_category_encode(history):
    history['orderTime'] = pd.to_datetime(history['orderTime'], unit='s')

    # 训练集和测试集最后一天是 2017-09-11
    now = datetime.datetime(2017, 9, 12)
    history['days_from_now'] = history['orderTime'].map(lambda order: (now - order).days)
    history['order_year'] = history['orderTime'].dt.year
    history['order_month'] = history['orderTime'].dt.month
    history['order_year_month'] = history['order_year'] * 100 + history['order_month']
    history['order_day'] = history['orderTime'].dt.day
    history['order_weekofyear'] = history['orderTime'].dt.weekofyear
    history['order_weekday'] = history['orderTime'].dt.weekday
    history['order_hour'] = history['orderTime'].dt.hour
    history['order_minute'] = history['orderTime'].dt.minute
    history['order_is_weekend'] = history['orderTime'].map(lambda d: 1 if (d == 0) | (d == 6) else 0)
    history['order_week_hour'] = history['order_weekday'] * 24 + history['order_hour']
    # 按照时间排序
    history = history.sort_values(by='orderTime')
    history.reset_index(drop=True, inplace=True)
    history = order_last_num(history)
    history = days_since_prior_order(history)

    history['continent'] = history['continent'].map(lambda c: '_'.join(lazy_pinyin(c)) if c == c else 'None')
    history['country'] = history['country'].map(lambda c: '_'.join(lazy_pinyin(c)) if c == c else 'None')
    history['city'] = history['city'].map(lambda c: '_'.join(lazy_pinyin(c)) if c == c else 'None')

    le = LabelEncoder()
    le.fit(history['continent'].values)
    history['continent'] = le.transform(history['continent'])
    le = LabelEncoder()
    le.fit(history['country'].values)
    history['country'] = le.transform(history['country'])
    le = LabelEncoder()
    le.fit(history['city'].values)
    history['city'] = le.transform(history['city'])

    return history

In [55]:
orderHistory_train = build_time_category_encode(orderHistory_train)
orderHistory_test = build_time_category_encode(orderHistory_test)
orderHistory_train.to_csv(Configure.cleaned_path + 'cleaned_orderHistory_train.csv', index=False,
                          columns=orderHistory_train.columns)
orderHistory_test.to_csv(Configure.cleaned_path + 'cleaned_orderHistory_test.csv', index=False,
                         columns=orderHistory_test.columns)

100%|██████████| 1055/1055 [00:09<00:00, 114.43it/s]


In [56]:
feature_name = 'user_order_history_features'
is_feature_created(feature_name)

False

In [66]:
def check_last_time_order_info(uid, userid_grouped, flag, check_name, last_time=1):
    """ 最近的一次交易的具体信息 check_name """
    if flag == 0:
        return -1

    df = userid_grouped[uid]
    if df.shape[0] < last_time:
        return -1
    else:
        return df.iloc[-last_time][check_name]
def pre_days_order_count(uid, userid_grouped, flag, days):
    """ 往前 days 的 order 数量 """
    if flag == 0:
        return 0

    df = userid_grouped[uid]
    df = df.loc[df['days_from_now'] < days]
    return df.shape[0]

def pre_days_checkname_diff_count(uid, userid_grouped, flag, days, check_name):
    """ 往前 days 的 order 的不同 check_name 数量 """
    if flag == 0:
        return 0

    df = userid_grouped[uid]
    df = df.loc[df['days_from_now'] < days]
    if df.shape[0] == 0:
        return 0
    else:
        return len(df[check_name].unique())


def year_order_count(uid, userid_grouped, flag, year):
    """ 2016年的 order 的不同 check_name 数量 """
    if flag == 0:
        return 0

    df = userid_grouped[uid]
    df = df.loc[df['order_year'] == year]
    return df.shape[0]


def year_checkname_diff_count(uid, userid_grouped, flag, year, check_name):
    """ year 的 order 的不同 check_name 数量 """
    if flag == 0:
        return 0

    df = userid_grouped[uid]
    df = df.loc[df['order_year'] == year]
    if df.shape[0] == 0:
        return 0
    else:
        return len(df[check_name].unique())


def year_order_month_count(uid, userid_grouped, flag, year):
    """ 每年去了几个月份 """
    if flag == 0:
        return 0

    df = userid_grouped[uid]
    df = df.loc[df['order_year'] == year]
    if df.shape[0] == 0:
        return 0
    else:
        return len(df['order_month'].unique())


def year_order_month_most(uid, userid_grouped, flag, year):
    """ 每年一个月去的最多的次数 """
    if flag == 0:
        return 0

    df = userid_grouped[uid]
    df = df.loc[df['order_year'] == year]
    df = df.groupby(['order_month']).count()['orderTime'].reset_index()
    if df.shape[0] == 0:
        return 0
    else:
        return df['orderTime'].max()


def year_most_order_month(uid, userid_grouped, flag, year):
    """ 每年去的最多次数的月份 """
    if flag == 0:
        return -1

    df = userid_grouped[uid]
    df = df.loc[df['order_year'] == year]
    df = df.groupby(['order_month']).count()['orderTime'].reset_index()
    if df.shape[0] == 0:
        return -1
    else:
        return df.sort_values(by='orderTime', ascending=False)['order_month'].values[0]


def year_good_order_count(uid, userid_grouped, flag, year):
    """ 每年精品订单数量 """
    if flag == 0:
        return 0

    df = userid_grouped[uid]
    df = df.loc[df['order_year'] == year]
    return sum(df['orderType'])

In [67]:
def build_order_history_features(df, history):
    features = pd.DataFrame({'userid': df['userid']})

    df_ids = history['userid'].unique()
    userid_grouped = dict(list(history.groupby('userid')))

    #给trade表打标签，若id在login表中，则打标签为1，否则为0
    features['has_history_flag'] = features['userid'].map(lambda uid: uid in df_ids).astype(int)

    print("基本特征")
    # 最近的一次交易的 days_from_now, order_year, order_month, order_day, order_weekofyear, order_weekday
    features['last_time_days_from_now'] = features.apply(lambda row: check_last_time_order_info(row['userid'], userid_grouped, row['has_history_flag'], 'days_from_now'), axis=1)
    features['last_time_order_year'] = features.apply(lambda row: check_last_time_order_info(row['userid'], userid_grouped, row['has_history_flag'], 'order_year'), axis=1)
    features['last_time_order_month'] = features.apply(lambda row: check_last_time_order_info(row['userid'], userid_grouped, row['has_history_flag'], 'order_month'), axis=1)
    features['last_time_order_day'] = features.apply(lambda row: check_last_time_order_info(row['userid'], userid_grouped, row['has_history_flag'], 'order_day'), axis=1)
    features['last_time_order_weekofyear'] = features.apply(lambda row: check_last_time_order_info(row['userid'], userid_grouped, row['has_history_flag'], 'order_weekofyear'), axis=1)
    features['last_time_order_weekday'] = features.apply(lambda row: check_last_time_order_info(row['userid'], userid_grouped, row['has_history_flag'], 'order_weekday'), axis=1)
    features['last_time_continent'] = features.apply(lambda row: check_last_time_order_info(row['userid'], userid_grouped, row['has_history_flag'], 'continent'), axis=1)
    features['last_time_country'] = features.apply(lambda row: check_last_time_order_info(row['userid'], userid_grouped, row['has_history_flag'], 'country'), axis=1)
    features['last_time_city'] = features.apply(lambda row: check_last_time_order_info(row['userid'], userid_grouped, row['has_history_flag'], 'city'), axis=1)

    print("计数特征")
    # 往前 90days 的计数特征
    features['pre_90days_order_count'] = features.apply(lambda row: pre_days_order_count(row['userid'], userid_grouped, row['has_history_flag'], 90), axis=1)
    features['pre_90days_order_continent_count'] = features.apply(lambda row: pre_days_checkname_diff_count(row['userid'], userid_grouped, row['has_history_flag'], 90, 'continent'), axis=1)
    features['pre_90days_order_country_count'] = features.apply(lambda row: pre_days_checkname_diff_count(row['userid'], userid_grouped, row['has_history_flag'], 90, 'country'), axis=1)
    features['pre_90days_order_city_count'] = features.apply(lambda row: pre_days_checkname_diff_count(row['userid'], userid_grouped, row['has_history_flag'], 90, 'city'), axis=1)

    features['2016_order_count'] = features.apply(lambda row: year_order_count(row['userid'], userid_grouped, row['has_history_flag'], 2016), axis=1)
    features['2017_order_count'] = features.apply(lambda row: year_order_count(row['userid'], userid_grouped, row['has_history_flag'], 2017), axis=1)
    features['2017_order_continent_count'] = features.apply(lambda row: year_checkname_diff_count(row['userid'], userid_grouped, row['has_history_flag'], 2017, 'continent'), axis=1)
    features['2017_order_country_count'] = features.apply(lambda row: year_checkname_diff_count(row['userid'], userid_grouped, row['has_history_flag'], 2017, 'country'), axis=1)
    features['2017_order_city_count'] = features.apply(lambda row: year_checkname_diff_count(row['userid'], userid_grouped, row['has_history_flag'], 2017, 'city'), axis=1)
    # 是否 2016 年和 2017 年都有 order
    features['both_year_has_order'] = features.apply(lambda row: (row['2016_order_count'] > 0) & (row['2017_order_count'] > 0), axis=1).astype(int)
    # 每年去了几个月份
    features['2016_order_month_count'] = features.apply(lambda row: year_order_month_count(row['userid'], userid_grouped, row['has_history_flag'], 2016), axis=1)
    features['2017_order_month_count'] = features.apply(lambda row: year_order_month_count(row['userid'], userid_grouped, row['has_history_flag'], 2017), axis=1)
    print('比率特征')
    # 用户总订单数、精品订单数、精品订单比例
    features['2016_good_order_count'] = features.apply(lambda row: year_good_order_count(row['userid'], userid_grouped, row['has_history_flag'], 2016), axis=1)
    features['2016_good_order_ratio'] = features.apply(lambda row: row['2016_good_order_count'] / row['2016_order_count'] if row['2016_order_count'] != 0 else 0, axis=1)

    features['2017_good_order_count'] = features.apply(lambda row: year_good_order_count(row['userid'], userid_grouped, row['has_history_flag'], 2017), axis=1)
    features['2017_good_order_ratio'] = features.apply(lambda row: row['2017_good_order_count'] / row['2017_order_count'] if row['2017_order_count'] != 0 else 0, axis=1)

    features['total_order_count'] = features['2016_order_count'] + features['2017_order_count']
    features['total_good_order_count'] = features['2016_good_order_count'] + features['2017_good_order_count']
    features['total_good_order_ratio'] = features.apply(lambda row: row['total_good_order_count'] / row['total_order_count'] if row['total_order_count'] != 0 else 0, axis=1)
    # has_good_order 强特
    features['has_good_order'] = (features['total_good_order_ratio'] > 0).astype(int)
    features.drop(['2016_good_order_count', '2017_good_order_count', 'total_order_count', 'total_good_order_count'], axis=1, inplace=True)
    return features

In [68]:
#build train user_order_history_features
train_features = build_order_history_features(train, orderHistory_train)
#build test user_order_history_features
test_features = build_order_history_features(test, orderHistory_test)
#print('save ', feature_name)
save_features(train_features, test_features, feature_name)

基本特征
计数特征
比率特征
基本特征
计数特征
比率特征


In [69]:
feature_name = 'user_order_history_features2'
is_feature_created(feature_name)

False

In [70]:
def year_first_last_order_history_type(uid, history_grouped, flag, year=2017):
    """  每年第一次和最后一次订单的 ordertype """
    if flag == 0:
        return -1, -1

    df = history_grouped[uid]
    df = df[df['order_year'] == year]

    if df.shape[0] < 1:
        return -1, -1

    first1_ordertype = df['orderType'].iat[0]
    last1_ordertype = df['orderType'].iat[-1]
    return first1_ordertype, last1_ordertype

In [71]:
def build_order_history_features2(df, history):
    features = pd.DataFrame({'userid': df['userid']})

    history_uids = history['userid'].unique()
    history_grouped = dict(list(history.groupby('userid')))

    #给trade表打标签，若id在login表中，则打标签为1，否则为0
    features['has_history_flag'] = features['userid'].map(lambda uid: uid in history_uids).astype(int)

    # # 子父订单统计特征
    # features['father_son_order_statistic'] = features.apply(lambda row: father_son_order_statistic(row['userid'], history_grouped, row['has_history_flag']), axis=1)
    # # features['has_father_son_order'] = features['father_son_order_statistic'].map(lambda x: x[0])
    # # features['father_son_order_order_type0_count'] = features['father_son_order_statistic'].map(lambda x: x[1])
    # # features['father_son_order_order_type1_count'] = features['father_son_order_statistic'].map(lambda x: x[2])
    # features['father_son_order_order_type0_ratio'] = features['father_son_order_statistic'].map(lambda x: x[0])
    # features['father_son_order_order_type1_ratio'] = features['father_son_order_statistic'].map(lambda x: x[1])
    # del features['father_son_order_statistic']

    print('强特:2016_2017_first_last_ordertype')
    features['2017_first_last_order_history_type'] = features.apply(lambda row: year_first_last_order_history_type(row['userid'], history_grouped, row['has_history_flag'], year=2017), axis=1)
    features['2017_first_order_history_type'] = features['2017_first_last_order_history_type'].map(lambda x: x[0])
    features['2017_last_order_history_type'] = features['2017_first_last_order_history_type'].map(lambda x: x[1])
    features['2016_first_last_order_history_type'] = features.apply(lambda row: year_first_last_order_history_type(row['userid'], history_grouped, row['has_history_flag'], year=2016), axis=1)
    features['2016_first_order_history_type'] = features['2016_first_last_order_history_type'].map(lambda x: x[0])
    features['2016_last_order_history_type'] = features['2016_first_last_order_history_type'].map(lambda x: x[1])

    features['2016_2017_first_last_ordertype'] = ((features['2016_first_order_history_type'] == 1) | (features['2017_first_order_history_type'] == 1) |
                                                  (features['2016_last_order_history_type'] == 1) | (features['2017_last_order_history_type'] == 1)).astype(int)

    features.drop(['2017_first_last_order_history_type', '2017_first_order_history_type', '2017_last_order_history_type',
                   '2016_first_last_order_history_type', '2016_first_order_history_type', '2016_last_order_history_type'], axis=1, inplace=True)

    print('每年每个月份订单的统计')
    df = history.groupby(by=['userid', 'order_year_month']).count().reset_index()[['userid', 'order_year_month', 'orderid']].rename(columns={'orderid': 'year_month_order_count'})
    df = df.pivot('userid', 'order_year_month', 'year_month_order_count').reset_index().fillna(0)
    df.columns = df.columns.astype(str)
    df.drop(['201709', '201708', '201707', '201701', '201705'], axis=1, inplace=True)
    features = features.merge(df, on='userid', how='left')
    del features['has_history_flag']
    return features

In [73]:
#build train user_order_history_features2
train_features = build_order_history_features2(train, orderHistory_train)
#build test user_order_history_features2
test_features = build_order_history_features2(test, orderHistory_test)
#print('save ', feature_name)
save_features(train_features, test_features, feature_name)

强特:2016_2017_first_last_ordertype
每年每个月份订单的统计
强特:2016_2017_first_last_ordertype
每年每个月份订单的统计


In [74]:
feature_name = 'user_order_history_features3'
is_feature_created(feature_name)

False

In [75]:
def last_order_location_hot(uid, history_grouped, flag, hot_df, column):
    if flag == 0:
        return -1

    df = history_grouped[uid]
    last_hot = hot_df[hot_df[column] == df[column].iat[-1]]['hot'].values[0]
    return last_hot

In [76]:
def build_order_history_features3(df, orderHistory, history):
    """ 热度分析 """
    city_hot = orderHistory.groupby(['city']).count()['userid'].reset_index().rename(columns={'userid': 'hot'})
    city_hot['hot'] = city_hot['hot'].astype(float) / sum(city_hot['hot'])
    country_hot = orderHistory.groupby(['country']).count()['userid'].reset_index().rename(columns={'userid': 'hot'})
    country_hot['hot'] = country_hot['hot'].astype(float) / sum(country_hot['hot'])
    continent_hot = orderHistory.groupby(['continent']).count()['userid'].reset_index().rename(columns={'userid': 'hot'})
    continent_hot['hot'] = continent_hot['hot'].astype(float) / sum(continent_hot['hot'])

    features = pd.DataFrame({'userid': df['userid']})
    history_uids = history['userid'].unique()
    history_grouped = dict(list(history.groupby('userid')))
    #给trade表打标签，若id在login表中，则打标签为1，否则为0
    features['has_history_flag'] = features['userid'].map(lambda uid: uid in history_uids).astype(int)


    # 只有 last_order_city_hot 线上A榜提升
    features['last_order_city_hot'] = features.apply(lambda row: last_order_location_hot(row['userid'], history_grouped, row['has_history_flag'], city_hot, 'city'), axis=1)
    # features['last_order_country_hot'] = features.apply(lambda row: last_order_location_hot(row['userid'], history_grouped, row['has_history_flag'], country_hot, 'country'), axis=1)
    # features['last_order_continent_hot'] = features.apply(lambda row: last_order_location_hot(row['userid'], history_grouped, row['has_history_flag'], continent_hot, 'continent'), axis=1)

    del features['has_history_flag']
    return features

In [77]:
orderHistory = pd.concat([orderHistory_train, orderHistory_test])
#build train user_order_history_features3
train_features = build_order_history_features3(train, orderHistory, orderHistory_train)
#build test user_order_history_features3
test_features = build_order_history_features3(test, orderHistory, orderHistory_test)
#print('save ', feature_name)
save_features(train_features, test_features, feature_name)


In [78]:
feature_name = 'user_order_history_features4'
is_feature_created(feature_name)

False

In [79]:
def build_order_history_features4(df, history):
    features = pd.DataFrame({'userid': df['userid']})

    history_uids = history['userid'].unique()
    history_grouped = dict(list(history.groupby('userid')))

    #给trade表打标签，若id在login表中，则打标签为1，否则为0
    features['has_history_flag'] = features['userid'].map(lambda uid: uid in history_uids).astype(int)

    # 多次订单并且有精品的老用户
    features['multi_order_has_good_order'] = features.apply(lambda row: multi_order_has_good_order(row['userid'], history_grouped, row['has_history_flag']), axis=1)

    # 多次订单并且都没有精品订单
    del features['has_history_flag']
    return features

In [82]:
def multi_order_has_good_order(uid, history_grouped, flag):
    """ 多次订单并且有精品的老用户 """
    if flag == 0:
        return 0

    df = history_grouped[uid]
    if (df.shape[0] > 1) and sum(df['orderType']) > 0:
        return 1

    return 0

In [83]:
#build train user_order_history_features4
train_features = build_order_history_features4(train, orderHistory_train)
#build test user_order_history_features4
test_features = build_order_history_features4(test, orderHistory_test)
print('save ', feature_name)
save_features(train_features, test_features, feature_name)

save  user_order_history_features4


In [84]:
feature_name = 'user_order_history_features_wxr'
is_feature_created(feature_name)

False

In [85]:
def build_order_history_features_wxr(df, orderHistory, history):
    features = pd.DataFrame({'userid': df['userid']})

    #用户平均每次旅行间隔
    predict_time = pd.to_datetime(datetime.datetime(2017, 9, 12))
    features['days_since_last_order'] = -1
    for i in tqdm(range(features.shape[0])):
        df = history[(history['userid'] == features.loc[i, 'userid']) & (history['order_number'] == 1)]
        if df.shape[0] > 0:
            features.loc[i, 'days_since_last_order'] = ((predict_time - df['orderTime'].values[0]) / np.timedelta64(1,'s')) / (24 * 3600.0)
    history_s = history.groupby(['userid'])['days_since_prior_order'].mean().reset_index()
    history_s.columns = ['userid', 'avg_days_between_order']
    features = pd.merge(features, history_s, on='userid', how='left')
    features['days_ratio_since_last_order'] = features['days_since_last_order'] / features[
        'avg_days_between_order']
    features['avg_days_between_order'] = features['avg_days_between_order'].fillna(-1)
    features['days_ratio_since_last_order'] = features['days_ratio_since_last_order'].fillna(-1)
    features.loc[features[features['avg_days_between_order'] == 0].index, 'days_ratio_since_last_order'] = 0
    del features['days_since_last_order']

    #用户去过的城市、国家、大陆数
    history_s = history.groupby(['userid', 'city']).size().reset_index()
    history_s = history_s.groupby(['userid']).size().reset_index()
    history_s.columns = ['userid', 'city_num']
    features = pd.merge(features, history_s, on='userid', how='left')
    features['city_num'] = features['city_num'].fillna(0).astype(int)

    history_s = history.groupby(['userid', 'country']).size().reset_index()
    history_s = history_s.groupby(['userid']).size().reset_index()
    history_s.columns = ['userid', 'country_num']
    features = pd.merge(features, history_s, on='userid', how='left')
    features['country_num'] = features['country_num'].fillna(0).astype(int)

    history_s = history.groupby(['userid', 'continent']).size().reset_index()
    history_s = history_s.groupby(['userid']).size().reset_index()
    history_s.columns = ['userid', 'continent_num']
    features = pd.merge(features, history_s, on='userid', how='left')
    features['continent_num'] = features['continent_num'].fillna(0).astype(int)

    #根据用户去的城市、国家给用户的富裕程度打分
    city_df = city_info(orderHistory)
    country_df = country_info(orderHistory)
    features = user_city_info(features, history, city_df)
    features = user_country_info(features, history, country_df)

    return features

In [86]:
def city_info(df):
    """ 城市订单特征 """
    df_select = df.groupby(['city']).size().reset_index()
    df_select.columns = ['city', 'city_order_num']
    df_select['city_order_ratio'] = df_select['city_order_num'] / (1.0 * df.shape[0])
    df_select_1 = df[df['orderType'] == 1].groupby(['city']).size().reset_index()
    df_select_1.columns = ['city', 'city_high_order_num']
    df_select = pd.merge(df_select, df_select_1, on='city', how='left')
    df_select['city_high_order_num'] = df_select['city_high_order_num'].fillna(0).astype(int)
    df_select['city_high_order_ratio'] = df_select['city_high_order_num'] / (1.0 * df_select['city_order_num'])

    del df_select_1
    return df_select


def country_info(df):
    """ 国家订单特征 """
    df_select = df.groupby(['country']).size().reset_index()
    df_select.columns = ['country', 'country_order_num']
    df_select['country_order_ratio'] = df_select['country_order_num'] / (1.0 * df.shape[0])
    df_select_1 = df[df['orderType'] == 1].groupby(['country']).size().reset_index()
    df_select_1.columns = ['country', 'country_high_order_num']
    df_select = pd.merge(df_select, df_select_1, on='country', how='left')
    df_select['country_high_order_num'] = df_select['country_high_order_num'].fillna(0).astype(int)
    df_select['country_high_order_ratio'] = df_select['country_high_order_num'] / (1.0 * df_select['country_order_num'])

    del df_select_1
    return df_select

def user_city_info(feature, df, info):
    """ 根据城市对用户富裕程度打分 """
    feature_c = feature.copy()
    #feature_c['city_popular'] = np.nan
    feature_c['city_rich'] = 0
    feature_c['city_avg_rich'] = 0
    for i in tqdm(range(feature_c.shape[0])):
        df_select = df[df['userid'] == feature_c.loc[i,'userid']].reset_index(drop=True)
        num = df_select.shape[0]
        #popular = 0
        rich = 0
        if num > 0:
            for j in range(num):
                info_select = info[info['city'] == df_select.loc[j,'city']]
                #popular += info_select['city_order_ratio'].values
                rich += (1-info_select['city_high_order_ratio'].values) if df_select.loc[j,'orderType'] == 1 else -1*info_select['city_high_order_ratio'].values
            #feature_c.loc[i, 'city_popular'] = popular/(1.0*num)
            feature_c.loc[i, 'city_rich'] = rich
            feature_c.loc[i, 'city_avg_rich'] = rich/(1.0*num)
        del df_select
    return feature_c

def user_country_info(feature, df, info):
    """ 根据国家对用户富裕程度打分 """
    feature_c = feature.copy()
    #feature_c['country_popular'] = np.nan
    feature_c['country_rich'] = 0
    feature_c['country_avg_rich'] = 0
    for i in tqdm(range(feature_c.shape[0])):
        df_select = df[df['userid'] == feature_c.loc[i,'userid']].reset_index(drop=True)
        num = df_select.shape[0]
        #popular = 0
        rich = 0
        if num > 0:
            for j in range(num):
                info_select = info[info['country'] == df_select.loc[j,'country']]
                #popular += info_select['country_order_ratio'].values
                rich += (1-info_select['country_high_order_ratio'].values) if df_select.loc[j,'orderType'] == 1 else -1*info_select['country_high_order_ratio'].values
            #feature_c.loc[i, 'country_popular'] = popular/(1.0*num)
            feature_c.loc[i, 'country_rich'] = rich
            feature_c.loc[i, 'country_avg_rich'] = rich/(1.0*num)
        del df_select
    return feature_c

In [87]:
orderHistory = pd.concat([orderHistory_train, orderHistory_test])
#build train user_order_history_features3
train_features = build_order_history_features_wxr(train, orderHistory, orderHistory_train)
#build test user_order_history_features3
test_features = build_order_history_features_wxr(test, orderHistory, orderHistory_test)
print('save ', feature_name)
save_features(train_features, test_features, feature_name)


100%|██████████| 4031/4031 [00:26<00:00, 153.57it/s]


save  user_order_history_features_wxr


In [88]:
feature_name = 'user_order_comment_features'
is_feature_created(feature_name)

False

In [90]:
train = pd.read_csv(Configure.raw_data_path + 'train/orderFuture_train.csv', encoding='utf8', dtype={"userid":str})
test = pd.read_csv(Configure.raw_data_path + 'test/orderFuture_test.csv', encoding='utf8', dtype={"userid":str})
userComment_train = pd.read_csv(Configure.raw_data_path + 'train/userComment_train.csv', encoding='utf8', dtype={"userid":str})
userComment_test = pd.read_csv(Configure.raw_data_path + 'test/userComment_test.csv', encoding='utf8', dtype={"userid":str})

In [91]:
userComment_train.loc[userComment_train['rating'] == 4.33, 'rating'] = 4
userComment_train.loc[userComment_train['rating'] == 3.67, 'rating'] = 4
userComment_test.loc[userComment_train['rating'] == 2.33, 'rating'] = 2
userComment_train['rating'] = userComment_train['rating'].astype(int)
userComment_test['rating'] = userComment_test['rating'].astype(int)

In [92]:
orderHistory_train = pd.read_csv(Configure.cleaned_path + 'cleaned_orderHistory_train.csv', encoding='utf8')
orderHistory_test = pd.read_csv(Configure.cleaned_path + 'cleaned_orderHistory_test.csv', encoding='utf8')
userComment_train = pd.merge(userComment_train, orderHistory_train[['orderid', 'orderType']], on='orderid', how='left')
userComment_test = pd.merge(userComment_test, orderHistory_test[['orderid', 'orderType']], on='orderid', how='left')

In [96]:
from aip import AipNlp
def commentKey_score(df):
    """使用百度情感分析API对用户评论打分,叫AipNlp"""
    APP_ID = '23049729'
    API_KEY = 'IyeaSnLGvEIXaW9nztxzH71V'
    SECRET_KEY = 'w7GyCZ9WOva3hVmGTlnlaIkIapH26lMr'
    df_c = df.copy()
    df_c['commentsKeyWords_score'] = np.nan
    df_select_index = df_c[pd.notnull(df_c['commentsKeyWords'])].index
    client = AipNlp(APP_ID, API_KEY, SECRET_KEY)
    # python2里的写法
    # for i in xrange(len(df_select_index)):
    for i in range(len(df_select_index)):
        comment_str = df_c.loc[df_select_index[i], 'commentsKeyWords']
        try:
            # 调用接口，发起请求
            res = client.sentimentClassify(comment_str)
            print(res)
            df_c.loc[df_select_index[i], 'commentsKeyWords_score'] = res['items'][0]['positive_prob']
            time.sleep(0.8)
        except Exception as e:
            print('exception:', e)

    return df_c

In [97]:
userComment_train = commentKey_score(userComment_train)
userComment_test = commentKey_score(userComment_test)

{'log_id': 7634579711558191686, 'text': "['很','简陋','太','随便']", 'items': [{'positive_prob': 0.00109093, 'confidence': 0.997576, 'negative_prob': 0.998909, 'sentiment': 0}]}
{'log_id': 1586269554791595942, 'text': "['很','细心']", 'items': [{'positive_prob': 0.995574, 'confidence': 0.990164, 'negative_prob': 0.00442636, 'sentiment': 2}]}
{'log_id': 95145856298921830, 'text': "['真心','好','惊喜','惊喜','合理','还','真心','赞']", 'items': [{'positive_prob': 0.999857, 'confidence': 0.999683, 'negative_prob': 0.000142699, 'sentiment': 2}]}
{'log_id': 6933660309545991334, 'text': "['非常','满意','认真','负责','诚恳','合理','很','精致','希望']", 'items': [{'positive_prob': 0.998589, 'confidence': 0.996865, 'negative_prob': 0.0014109, 'sentiment': 2}]}
{'log_id': 6241926572040789894, 'text': "['very','good']", 'items': [{'positive_prob': 0.999605, 'confidence': 0.999122, 'negative_prob': 0.00039504, 'sentiment': 2}]}
{'log_id': 9116086524592991878, 'text': "['很','负责','辛苦','还']", 'items': [{'positive_prob': 0.945405, 'confiden

{'log_id': 3919767817862637126, 'text': "['热情','细致','很','详细','还','注意']", 'items': [{'positive_prob': 0.967354, 'confidence': 0.927454, 'negative_prob': 0.0326455, 'sentiment': 2}]}
{'log_id': 7348277775794255014, 'text': "['非常','好']", 'items': [{'positive_prob': 0.995987, 'confidence': 0.991083, 'negative_prob': 0.00401266, 'sentiment': 2}]}
{'log_id': 1876625851136131558, 'text': "['标准','娴熟']", 'items': [{'positive_prob': 0.982023, 'confidence': 0.960052, 'negative_prob': 0.0179766, 'sentiment': 2}]}
{'log_id': 2053650478377410918, 'text': "['很','好','很','好','很','好','很','好']", 'items': [{'positive_prob': 0.998156, 'confidence': 0.995902, 'negative_prob': 0.00184424, 'sentiment': 2}]}
{'log_id': 4600132512380814822, 'text': "['很','不错','稳定','还','很','好','还','很','还','怕','很','一定']", 'items': [{'positive_prob': 0.994399, 'confidence': 0.987553, 'negative_prob': 0.00560118, 'sentiment': 2}]}
{'log_id': 8642585685996660166, 'text': "['很','不错','辛苦']", 'items': [{'positive_prob': 0.999341, 'conf

{'log_id': 9206316470444472806, 'text': "['热情','大方','礼貌','非常','周到']", 'items': [{'positive_prob': 0.997839, 'confidence': 0.995198, 'negative_prob': 0.00216087, 'sentiment': 2}]}
{'log_id': 8834503981315293030, 'text': "['很','不错']", 'items': [{'positive_prob': 0.998717, 'confidence': 0.997148, 'negative_prob': 0.00128339, 'sentiment': 2}]}
{'log_id': 7032548050494111430, 'text': "['很','好']", 'items': [{'positive_prob': 0.996388, 'confidence': 0.991974, 'negative_prob': 0.00361147, 'sentiment': 2}]}
{'log_id': 4246180090219972134, 'text': "['非常','满意','好']", 'items': [{'positive_prob': 0.998922, 'confidence': 0.997605, 'negative_prob': 0.00107787, 'sentiment': 2}]}
{'log_id': 5797938519629449446, 'text': "['非常热心','热情','顺利','还','热情','非常','满意']", 'items': [{'positive_prob': 0.999641, 'confidence': 0.999202, 'negative_prob': 0.000359198, 'sentiment': 2}]}
{'log_id': 4314362478281018534, 'text': "['非常','好','好']", 'items': [{'positive_prob': 0.997395, 'confidence': 0.994212, 'negative_prob': 

{'log_id': 4572144877032072870, 'text': "['非常','清楚','很','顺利','惊喜','热心','简单','莫大','很','幸运']", 'items': [{'positive_prob': 0.998622, 'confidence': 0.996939, 'negative_prob': 0.00137757, 'sentiment': 2}]}
{'log_id': 3843172939702020262, 'text': "['很','满意']", 'items': [{'positive_prob': 0.998636, 'confidence': 0.996968, 'negative_prob': 0.00136439, 'sentiment': 2}]}
{'log_id': 4221797890100765574, 'text': "['挺','好','安全','还','真心','感谢','十分']", 'items': [{'positive_prob': 0.97276, 'confidence': 0.939467, 'negative_prob': 0.02724, 'sentiment': 2}]}
{'log_id': 4076115581175872966, 'text': "['安全']", 'items': [{'positive_prob': 0.969062, 'confidence': 0.931249, 'negative_prob': 0.0309378, 'sentiment': 2}]}
{'log_id': 4518062063925279046, 'text': "['很','满意','很','热情','很','细心','方便']", 'items': [{'positive_prob': 0.998651, 'confidence': 0.997002, 'negative_prob': 0.00134914, 'sentiment': 2}]}
{'log_id': 70605106028856390, 'text': "['完全','非常','热情','很','有点','很','热情','还','非常','非常','满意','感谢','感谢']", 'ite

{'log_id': 1106450654814339494, 'text': "['确实','不错']", 'items': [{'positive_prob': 0.996953, 'confidence': 0.993228, 'negative_prob': 0.00304732, 'sentiment': 2}]}
{'log_id': 1684337195329932390, 'text': "['熟悉','非常','合理','还','非常','会','肯定','还']", 'items': [{'positive_prob': 0.977818, 'confidence': 0.950708, 'negative_prob': 0.0221815, 'sentiment': 2}]}
{'log_id': 6876621339479788198, 'text': "['非常','热心']", 'items': [{'positive_prob': 0.995362, 'confidence': 0.989692, 'negative_prob': 0.00463843, 'sentiment': 2}]}
{'log_id': 5813371800178750758, 'text': "['下','非常','陌生','迷茫','很']", 'items': [{'positive_prob': 0.863059, 'confidence': 0.695686, 'negative_prob': 0.136941, 'sentiment': 2}]}
{'log_id': 1667099688362870758, 'text': "['主动','聊','非常','愉快','祝','顺利']", 'items': [{'positive_prob': 0.995776, 'confidence': 0.990613, 'negative_prob': 0.00422416, 'sentiment': 2}]}
{'log_id': 5561903678818081702, 'text': "['很','周到','很','满意']", 'items': [{'positive_prob': 0.998583, 'confidence': 0.996851, 

...

{'log_id': 3029175994357855590, 'text': "['高','说好','帅']", 'items': [{'positive_prob': 0.999876, 'confidence': 0.999726, 'negative_prob': 0.000123541, 'sentiment': 2}]}
{'log_id': 1189668096190298790, 'text': "['特别','好']", 'items': [{'positive_prob': 0.997084, 'confidence': 0.993519, 'negative_prob': 0.00291646, 'sentiment': 2}]}
{'log_id': 6274961619043518150, 'text': "['很','好','还','超级','感谢']", 'items': [{'positive_prob': 0.994326, 'confidence': 0.987391, 'negative_prob': 0.00567419, 'sentiment': 2}]}
{'log_id': 5444818028608402534, 'text': '[]', 'items': [{'positive_prob': 0.841203, 'confidence': 0.647118, 'negative_prob': 0.158797, 'sentiment': 2}]}
{'log_id': 5460636674279613382, 'text': "['非常','热情']", 'items': [{'positive_prob': 0.992423, 'confidence': 0.983162, 'negative_prob': 0.0075769, 'sentiment': 2}]}
{'log_id': 602498543710601190, 'text': "['非常热心','很','好','非常','亲切']", 'items': [{'positive_prob': 0.999719, 'confidence': 0.999375, 'negative_prob': 0.000281065, 'sentiment': 2}]

{'log_id': 3873979481461566086, 'text': "['很','好','很','舒服']", 'items': [{'positive_prob': 0.998458, 'confidence': 0.996573, 'negative_prob': 0.00154192, 'sentiment': 2}]}
{'log_id': 2324654033575417382, 'text': "['很','好','准时','周到','热情']", 'items': [{'positive_prob': 0.99861, 'confidence': 0.996911, 'negative_prob': 0.00139017, 'sentiment': 2}]}
{'log_id': 1955243770376741798, 'text': "['非常','热情','很','一定']", 'items': [{'positive_prob': 0.992467, 'confidence': 0.983259, 'negative_prob': 0.00753341, 'sentiment': 2}]}
{'log_id': 9172420742416472550, 'text': "['热情','细致','干净','整洁','谢谢']", 'items': [{'positive_prob': 0.997725, 'confidence': 0.994944, 'negative_prob': 0.00227536, 'sentiment': 2}]}
{'log_id': 6178969765853334118, 'text': "['非常','主动']", 'items': [{'positive_prob': 0.982458, 'confidence': 0.961018, 'negative_prob': 0.017542, 'sentiment': 2}]}
{'log_id': 2292211913307252646, 'text': "['热情','彬彬有礼','赞','高','成','好']", 'items': [{'positive_prob': 0.99928, 'confidence': 0.9984, 'negati

{'log_id': 2094055115147408390, 'text': '[]', 'items': [{'positive_prob': 0.841203, 'confidence': 0.647118, 'negative_prob': 0.158797, 'sentiment': 2}]}
{'log_id': 5239221291461999974, 'text': "['满意']", 'items': [{'positive_prob': 0.998953, 'confidence': 0.997672, 'negative_prob': 0.00104738, 'sentiment': 2}]}
{'log_id': 8177330058378729382, 'text': "['非常','满意']", 'items': [{'positive_prob': 0.999089, 'confidence': 0.997975, 'negative_prob': 0.000911444, 'sentiment': 2}]}
{'log_id': 7863515838234427622, 'text': "['很','不错','热情','尤其','很','主动','上','下','很','上']", 'items': [{'positive_prob': 0.996758, 'confidence': 0.992796, 'negative_prob': 0.00324169, 'sentiment': 2}]}
{'log_id': 7241642092290498534, 'text': "['期待']", 'items': [{'positive_prob': 0.995858, 'confidence': 0.990795, 'negative_prob': 0.00414226, 'sentiment': 2}]}
{'log_id': 8280012644630253990, 'text': "['约','好']", 'items': [{'positive_prob': 0.997544, 'confidence': 0.994541, 'negative_prob': 0.00245636, 'sentiment': 2}]}
{'lo

{'log_id': 6634408120937227078, 'text': '[]', 'items': [{'positive_prob': 0.841203, 'confidence': 0.647118, 'negative_prob': 0.158797, 'sentiment': 2}]}
{'log_id': 8582099500396865478, 'text': "['非常','满意']", 'items': [{'positive_prob': 0.999089, 'confidence': 0.997975, 'negative_prob': 0.000911443, 'sentiment': 2}]}
{'log_id': 6322935035355987494, 'text': "['很','不错']", 'items': [{'positive_prob': 0.998717, 'confidence': 0.997148, 'negative_prob': 0.00128339, 'sentiment': 2}]}
{'log_id': 6720811834590380134, 'text': "['准时','蛮','谢谢','小','很','热情']", 'items': [{'positive_prob': 0.990953, 'confidence': 0.979895, 'negative_prob': 0.00904708, 'sentiment': 2}]}
{'log_id': 7648046018371272038, 'text': '[]', 'items': [{'positive_prob': 0.841203, 'confidence': 0.647118, 'negative_prob': 0.158797, 'sentiment': 2}]}
{'log_id': 4684125254783403462, 'text': "['非常','好','非常','热情']", 'items': [{'positive_prob': 0.998182, 'confidence': 0.995961, 'negative_prob': 0.00181777, 'sentiment': 2}]}
{'log_id': 6

{'log_id': 504166632107160998, 'text': "['非常','热情','主动','非常','合适','还','很','细心','注意']", 'items': [{'positive_prob': 0.988349, 'confidence': 0.974109, 'negative_prob': 0.0116511, 'sentiment': 2}]}
{'log_id': 664496804290088774, 'text': "['非常','满意']", 'items': [{'positive_prob': 0.999089, 'confidence': 0.997975, 'negative_prob': 0.000911443, 'sentiment': 2}]}
{'log_id': 2752086902528194662, 'text': "['很','顺利']", 'items': [{'positive_prob': 0.996877, 'confidence': 0.99306, 'negative_prob': 0.00312283, 'sentiment': 2}]}
{'log_id': 7006072603869783846, 'text': "['很','满意']", 'items': [{'positive_prob': 0.998636, 'confidence': 0.996968, 'negative_prob': 0.00136439, 'sentiment': 2}]}
{'log_id': 1556104702087662694, 'text': "['非常','好','干净','豪华','很','nice','愉快']", 'items': [{'positive_prob': 0.999684, 'confidence': 0.999298, 'negative_prob': 0.000315888, 'sentiment': 2}]}
{'log_id': 8123999294094727462, 'text': "['不错']", 'items': [{'positive_prob': 0.999032, 'confidence': 0.997848, 'negative_prob

In [98]:
def tag_score(df):
    """分正面、负面统计tag"""
    positive_tags = []
    negative_tags = []
    f = open('./tags.txt', 'r', encoding='utf-8')
    for line in f:
        arr = line.split()
        if arr[1] == '1':
            positive_tags.append(arr[0])
        else:
            negative_tags.append(arr[0])
    f.close()
    print (len(positive_tags), len(negative_tags))
    df_c = df.copy()
    df_c['positive_tags'] = 0
    df_c['negative_tags'] = 0
    for i in tqdm(range(df_c.shape[0])):
        tag_str = df_c.loc[i, 'tags']
        if tag_str == tag_str:
            tag_arr = tag_str.split(u'|')
            p_num = 0
            n_num = 0
            for tag in tag_arr:
                if tag in positive_tags:
                    p_num += 1
                elif tag in negative_tags:
                    n_num += 1
            df_c.loc[i, 'positive_tags'] = p_num
            df_c.loc[i, 'negative_tags'] = n_num
    df_c['tags_score'] = df_c['positive_tags'] - df_c['negative_tags']
    return df_c


In [102]:
userComment_train = tag_score(userComment_train)
userComment_test = tag_score(userComment_test)

  0%|          | 17/8871 [00:00<00:57, 153.82it/s]

16 33


  5%|▍         | 49/992 [00:00<00:01, 486.22it/s]

16 33


100%|██████████| 992/992 [00:01<00:00, 540.61it/s]


In [103]:
#save cleaned datasets
userComment_train.to_csv(Configure.cleaned_path + 'cleaned_userComment_train.csv', index=False, columns=userComment_train.columns, encoding='utf8')
userComment_test.to_csv(Configure.cleaned_path + 'cleaned_userComment_test.csv', index=False, columns=userComment_test.columns, encoding='utf8')


In [104]:
def built_comment_features(df, comments):
    features = pd.DataFrame({'userid': df['userid']})
    df_ids = comments['userid'].unique()
    userid_grouped = dict(list(comments.groupby('userid')))
    # 基本特征
    features['has_comment_flag'] = features['userid'].map(lambda uid: uid in df_ids).astype(int)
    features['user_rating_sattistic'] = features.apply(lambda row: user_rating_sattistic(row['userid'], userid_grouped, row['has_comment_flag']), axis=1)
    features['user_rating_std'] = features['user_rating_sattistic'].map(lambda x: x[4])
    # features['user_comment_count'] = features['user_rating_sattistic'].map(lambda x: x[5])
    del features['user_rating_sattistic']
    # 用户打分比例
    features['user_rating_ratio'] = features.apply(lambda row: user_rating_ratio(row['userid'], userid_grouped, row['has_comment_flag']), axis=1)
    features['raing1_ratio'] = features['user_rating_ratio'].map(lambda x: x[0])
    features['raing2_ratio'] = features['user_rating_ratio'].map(lambda x: x[1])
    features['raing3_ratio'] = features['user_rating_ratio'].map(lambda x: x[2])
    features['raing4_ratio'] = features['user_rating_ratio'].map(lambda x: x[3])
    # features['raing5_ratio'] = features['user_rating_ratio'].map(lambda x: x[4])
    del features['user_rating_ratio']
    return features

In [106]:
def user_rating_sattistic(uid, userid_grouped, flag):
    """ 用户订单的评论信息 """
    if flag == 0:
        return -1, -1, -1, -1, -1, -1

    df = userid_grouped[uid]
    if df.shape[0] == 0:
        return -1, -1, -1, -1, -1, -1
    else:
        return df['rating'].mean(), df['rating'].max(), df['rating'].min(), df['rating'].median(), df['rating'].std(), df.shape[0]

In [107]:
def user_rating_ratio(uid, userid_grouped, flag):
    """ 用户打分的比例 """
    if flag == 0:
        return -1, -1, -1, -1, -1

    df = userid_grouped[uid]
    if df.shape[0] == 0:
        return -1, -1, -1, -1, -1
    else:
        ratings = df['rating'].tolist()
        count_1 = float(ratings.count(1))
        count_2 = float(ratings.count(2))
        count_3 = float(ratings.count(3))
        count_4 = float(ratings.count(4))
        count_5 = float(ratings.count(5))
        return count_1 / df.shape[0], count_2 / df.shape[0], count_3 / df.shape[0], count_4 / df.shape[0], count_5 / df.shape[0]


In [108]:
#build train features
train_features = built_comment_features(train, userComment_train)
#'build test features
test_features = built_comment_features(test, userComment_test)
print('save ', feature_name)
save_features(train_features, test_features, feature_name)

save  user_order_comment_features


In [109]:
#build wxr features
feature_name = 'user_order_comment_features_wxr'
is_feature_created(feature_name)

False

In [110]:
def built_comment_features_wxr(df, comments, order):
    features = pd.DataFrame({'userid': df['userid']})
    features = avg_rating(features,comments)
    features = rating_last_order(features, comments, order)
    features = user_tag_score(features, comments)
    features = tag_last_order(features, comments, order)
    features = user_comment_score(features, comments)
    features = comment_last_order(features, comments, order)
    return features

In [111]:
def avg_rating(feature, comment):
    """用户全部订单平均分，普通订单平均分，精品订单平均分"""
    feature_c = feature.copy()
    df = comment.groupby(['userid'])['rating'].mean().reset_index()
    df.columns = ['userid', 'avg_rating']
    feature_c = pd.merge(feature_c, df, on='userid', how='left')
    feature_c['avg_rating'] = feature_c['avg_rating'].fillna(0)

    comment_c = comment[comment['orderType'] == 0]
    df = comment_c.groupby(['userid'])['rating'].mean().reset_index()
    df.columns = ['userid', 'avg_rating_type0']
    feature_c = pd.merge(feature_c, df, on='userid', how='left')
    feature_c['avg_rating_type0'] = feature_c['avg_rating_type0'].fillna(0)

    comment_c = comment[comment['orderType'] == 1]
    df = comment_c.groupby(['userid'])['rating'].mean().reset_index()
    df.columns = ['userid', 'avg_rating_type1']
    feature_c = pd.merge(feature_c, df, on='userid', how='left')
    feature_c['avg_rating_type1'] = feature_c['avg_rating_type1'].fillna(0)
    del df, comment_c
    return feature_c

def rating_last_order(feature, comment, order):
    """用户最近一次订单打分"""
    feature_c = feature.copy()
    feature_c['rating_last_order'] = 0
    for i in range(feature_c.shape[0]):
        userid = feature.loc[i,'userid']
        df = order[(order['userid'] == userid) & (order['order_number'] == 1)]
        if df.shape[0]>0:
            comment_df = comment[comment['orderid'] == df['orderid'].values[0]]
            if comment_df.shape[0]>0:
                feature_c.loc[i,'rating_last_order'] = comment_df['rating'].values[0]
    return feature_c

def user_tag_score(feature, df):
    """用户全部订单tag平均分"""
    feature_c = feature.copy()
    df_select = df.groupby(['userid'])['tags_score'].mean().reset_index()
    df_select.columns = ['userid','tags_score']
    feature_c = pd.merge(feature_c, df_select, on = 'userid', how = 'left')
    feature_c['tags_score'] = feature_c['tags_score'].fillna(0)
    del df_select
    return feature_c

def tag_last_order(feature, comment, order):
    """用户最近一次订单tag打分"""
    feature_c = feature.copy()
    feature_c['tags_score_last_order'] = 0
    for i in tqdm(range(feature_c.shape[0])):
        userid = feature.loc[i,'userid']
        df = order[(order['userid'] == userid) & (order['order_number'] == 1)]
        if df.shape[0]>0:
            comment_df = comment[comment['orderid'] == df['orderid'].values[0]]
            if comment_df.shape[0]>0:
                feature_c.loc[i,'tags_score_last_order'] = comment_df['tags_score'].values[0]
    return feature_c

def user_comment_score(feature, df):
    """用户全部commentkey平均分"""
    feature_c = feature.copy()
    df_select = df.groupby(['userid'])['commentsKeyWords_score'].mean().reset_index()
    df_select.columns = ['userid','commentsKeyWords_score']
    feature_c = pd.merge(feature_c, df_select, on = 'userid', how = 'left')
    feature_c['commentsKeyWords_score'] = feature_c['commentsKeyWords_score'].fillna(-1)
    del df_select
    return feature_c


def comment_last_order(feature, comment, order):
    """用户最近一次订单comment打分"""
    feature_c = feature.copy()
    feature_c['commentsKeyWords_score_last_order'] = -1
    for i in tqdm(range(feature_c.shape[0])):
        userid = feature.loc[i,'userid']
        df = order[(order['userid'] == userid) & (order['order_number'] == 1)]
        if df.shape[0]>0:
            comment_df = comment[comment['orderid'] == df['orderid'].values[0]]
            if comment_df.shape[0]>0:
                feature_c.loc[i,'commentsKeyWords_score_last_order'] = comment_df['commentsKeyWords_score'].values[0]
    return feature_c


In [112]:
#build train action history features11
train_features = built_comment_features_wxr(train, userComment_train,orderHistory_train)
#build test action history features11
test_features = built_comment_features_wxr(test, userComment_test,orderHistory_test)
print('save ', feature_name)
save_features(train_features, test_features, feature_name)

100%|██████████| 4031/4031 [00:47<00:00, 85.04it/s] 


save  user_order_comment_features_wxr
